# Calculating rate performance for multiple cells

## LIbrary imports and some default values for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter
import os
from ipywidgets import widgets
from ipywidgets import Button, HBox, VBox, Text, interactive_output

In [ ]:
markers = ['.', 'o', 's', 'v', '^', '<', '>', 'p', '*', 'D', 'd', 'H', 'h']
marker_fill = ['full', 'left', 'right', 'bottom', 'top', 'none']
normalization_options = ['Specific Capacitance (F/g)', 'Specific Capacity (mAh/g)', 'Specific Charge (C/g)', 
                         'Volumetric Capacitance (F/cm$^3$)', 'Volumetric Capacity (mAh/cm$^3$)',
                         'Volumetric Charge (C/cm$^3$)', 'Areal Capacitance (F/cm$^2$)', 
                         'Areal Capacity (mAh/cm$^2$)', 'Areal Charge (C/cm$^2$)']

## Reading in files, preprocessing data, and input of user constants

In [ ]:
# Enter path to folder containing your data in quotes.
rootdir = ''

In [ ]:
# Enter your scan rates as a comma separated list in the square brackets, e.g., [5, 10, 20, 50, 100].
scan_rates = []

In [ ]:
# Enter your values as a comma separated list in the square brackets. 
# Okay to leave empty if you don't have values.

electrode_mass = [] #in mg
electrode_area = [] #in cm^2
electrode_volume = [] #in cm^3

In [ ]:
def create_file_dict(rootdir, scan_rates):
    '''
    Creates a dictionary of file paths that correspond to the cell 
    data that will be processed. Files must be named such that they 
    are pre-sorted. Files are grouped by scan rate.
    '''

    file_dict = {}

    directory_path = [x[0] for x in os.walk(rootdir)][1:]
    file_path = [x[2] for x in os.walk(rootdir)][1:]

    full_path_list = []
    for idx, nested_list in enumerate(file_path):
        temp = []
        for element in nested_list:
            temp.append('{}\\{}'.format(directory_path[idx], element))
        full_path_list.append(temp)

    for scan_rate, path in zip(scan_rates, full_path_list):
        file_dict[scan_rate] = path

    return file_dict

In [ ]:
def rate_performance(data):
    '''
    Calculates average capacitance, capacity, and charge (and error values) 
    for a series of scan rates.Returns a plot of average capacitance with 
    error bars versus scan rate.
    '''
    potential_window = 0
    
    if potential_window == 0:
        potential_window += (max(data['Ewe/V']) - min(data['Ewe/V']))

    cycle_number = data['cycle number'].unique()

    charge_list = []
    charge_data = data[data['<I>/mA'] > 0]
    for n in cycle_number[1:-1]:
        charge_list.append(
            np.trapz(
            charge_data['<I>/mA'][charge_data['cycle number'] == n], 
            charge_data['time/s'][charge_data['cycle number'] == n]
            )
        )

    charge = np.mean(charge_list)
    charge_var = np.var(charge_list)

    discharge_list = []
    discharge_data = data[data['<I>/mA'] < 0]
    for n in cycle_number[1:-1]:
        discharge_list.append(
            np.trapz(
            discharge_data['<I>/mA'][discharge_data['cycle number'] == n], 
            discharge_data['time/s'][discharge_data['cycle number'] == n]
            )
        )

    discharge = np.abs(np.mean(discharge_list))
    discharge_var = np.abs(np.var(discharge_list))

    ce = np.mean([i / j for i, j in zip(np.abs(discharge_list), charge_list)])
    ce_var = np.var([i / j for i, j in zip(np.abs(discharge_list), charge_list)])
            
    return charge, discharge, charge_var, discharge_var, ce, ce_var, potential_window

In [ ]:
def create_data_dict(file_dict):
    '''
    Creates and then sorts a dictionary of the number of cycles,
    average potentials, average currents, and current variances 
    from the files listed in the file_dict. Entries are grouped
    by scan rate.
    '''

    data_dict = {}

    for key in file_dict:
        data_dict[key] = {'cycles': [],
                          'capacitance': [],
                          'capacitance_var': [],
                          'dis_capacitance': [],
                          'dis_capacitance_var': [],
                          'capacity': [],
                          'capacity_var': [],
                          'dis_capacity': [],
                          'dis_capacity_var': [],
                          'coulombs': [],
                          'coulombs_var': [],
                          'dis_coulombs': [],
                          'dis_coulombs_var': [],
                          'coulomb_eff': [],
                          'c_e_var': []
                          }

    for key in file_dict:

        for element in file_dict[key]:

            with open(element, 'r') as f:
                data = pd.read_table(f)
                cycles = max(data['cycle number']) - 2
                charge, discharge, charge_var, discharge_var, ce, ce_var, potential_window = rate_performance(data=data)

            data_dict[key]['cycles'].append(cycles)
            
            data_dict[key]['capacitance'].append(charge / potential_window)
            data_dict[key]['capacitance_var'].append(charge_var / potential_window)
            data_dict[key]['dis_capacitance'].append(discharge / potential_window)
            data_dict[key]['dis_capacitance_var'].append(discharge_var / potential_window)
            
            data_dict[key]['capacity'].append(charge / 3600)
            data_dict[key]['capacity_var'].append(charge_var / 3600)
            data_dict[key]['dis_capacity'].append(discharge / 3600)
            data_dict[key]['dis_capacity_var'].append(discharge_var / 3600)
            
            data_dict[key]['coulombs'].append(charge / 1000)
            data_dict[key]['coulombs_var'].append(charge_var / 1000)
            data_dict[key]['dis_coulombs'].append(discharge / 1000)
            data_dict[key]['dis_coulombs_var'].append(discharge_var / 1000)
            
            data_dict[key]['coulomb_eff'].append(ce)
            data_dict[key]['c_e_var'].append(ce_var)
   
    return data_dict

In [ ]:
def get_weighted_avgs_std(data_dict, scalar):
    '''
    Generates dictionary containing the weighted averages of the
    potenials and currents and the current standard deviation for
    each scan rate in the data_dict. Weighted averages use the
    number of cycles recorded at each scan rate
    '''

    scaled_charge_data = np.zeros(len(scan_rates))
    scaled_discharge_data = np.zeros(len(scan_rates))
    scaled_charge_data_std = np.zeros(len(scan_rates))
    scaled_discharge_data_std = np.zeros(len(scan_rates))
    weighted_c_eff = np.zeros(len(scan_rates))
    c_eff_std = np.zeros(len(scan_rates))
    
    for idx, key in enumerate(data_dict):
        for i in range(len(data_dict[key]['capacitance'])):
            if 'Capacitance' in scalar:
                if 'Specific' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['capacitance'][i] / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['capacitance_var'][i] / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_capacitance'][i] / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_capacitance_var'][i] / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Volumetric' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['capacitance'][i] / (electrode_volume[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['capacitance_var'][i] / (electrode_volume[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_capacitance'][i] / (electrode_volume[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_capacitance_var'][i] / (electrode_volume[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Areal' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['capacitance'][i] / (electrode_area[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['capacitance_var'][i] / (electrode_area[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_capacitance'][i] / (electrode_area[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_capacitance_var'][i] / (electrode_area[i] * 1000)) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

            elif "Capacity" in scalar:
                if 'Specific' in scalar:
                    scaled_charge_data[idx] += ((data_dict[key]['capacity'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += ((data_dict[key]['capacity_var'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += ((data_dict[key]['dis_capacity'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += ((data_dict[key]['dis_capacity_var'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Volumetric' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['capacity'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['capacity_var'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_capacity'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_capacity_var'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Areal' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['capacity'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['capacity_var'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_capacity'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_capacity_var'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

            elif 'Charge' in scalar:
                if 'Specific' in scalar:
                    scaled_charge_data[idx] += ((data_dict[key]['coulombs'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += ((data_dict[key]['coulombs_var'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += ((data_dict[key]['dis_coulombs'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += ((data_dict[key]['dis_coulombs_var'][i] * 1000) / electrode_mass[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Volumetric' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['coulombs'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['coulombs_var'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_coulombs'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_coulombs_var'][i] / electrode_volume[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

                elif 'Areal' in scalar:
                    scaled_charge_data[idx] += (data_dict[key]['coulombs'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_charge_data_std[idx] += (data_dict[key]['coulombs_var'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data[idx] += (data_dict[key]['dis_coulombs'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))
                    scaled_discharge_data_std[idx] += (data_dict[key]['dis_coulombs_var'][i] / electrode_area[i]) * (data_dict[key]['cycles'][i] / sum(data_dict[key]['cycles']))

    return (scaled_charge_data, scaled_discharge_data, 
            scaled_charge_data_std, scaled_discharge_data_std, 
            weighted_c_eff, c_eff_std)

## BioLogic rate performance

In [ ]:
def plot_rate_perf(yaxis, markers, marker_fill, color, c_e=False):
    
    global fig
    
    (scaled_charge_data, scaled_discharge_data, 
     scaled_charge_data_std, scaled_discharge_data_std, 
     weighted_c_eff, c_eff_std) = get_weighted_avgs_std(data_dict=data_dict, scalar=yaxis)
    
    fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
    
    ax.errorbar(scan_rates, scaled_discharge_data, yerr=scaled_charge_data_std,
            marker=markers, 
            fillstyle=marker_fill,
            markersize=12,
            color=color,
            capsize=5,
            capthick=2
           )

    ax.set_ylim(0, max(scaled_discharge_data) * 1.15)


    if c_e:
        ax2 = ax.twinx()
        ax2.errorbar(scan_rates, weighted_c_eff * 100, yerr=c_eff_std * 100,
                label='Coulombic efficiency',
                marker=markers,
                markersize=12,
                color='red', 
                capsize=5,
                capthick=2
                       )
        
        ax2.set_ylim(min(ce * 100) * 0.99, max(ce * 100) * 1.01)
        ax2.tick_params(which='both', labelsize=18, width=2, length=5)
        ax2.legend(frameon=False)

    ax.set_xlim(min(scan_rates) * 0.8 , max(scan_rates) * 1.5)
    ax.set_xlabel('Scan rate (mV/s)', fontsize=24)
    ax.set_ylabel(yaxis, fontsize=24)
    ax.tick_params(which='both', labelsize=18, width=2, length=5)
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
    ax.set_xticks(scan_rates)

In [ ]:
file_dict = create_file_dict(rootdir=rootdir, scan_rates=scan_rates)

In [ ]:
data_dict = create_data_dict(file_dict=file_dict)

In [ ]:
def make_figure():
    '''
    Generates interactive figure using the plot_CVs_biologic function.
    '''
    yaxis_choice = widgets.Dropdown(description='yaxis:', options=normalization_options)
    marker_choice = widgets.Dropdown(description='Markers:', options=markers)
    marker_fill_choice = widgets.Dropdown(description='Marker fill:', options=marker_fill)
    c_e = widgets.Checkbox(description='Coulombic Efficiency', indent=False)
    color_picker = widgets.ColorPicker(concise=False, description='Pick a color', value='blue', disabled=False)

    def on_button_clicked(b):
        fig.savefig('rate_performance', bbox_inches='tight', transparent=True)

    button = Button(description="Save Figure")
    button.on_click(on_button_clicked)

    ui = VBox([HBox([yaxis_choice, marker_choice, c_e, button]), HBox([marker_fill_choice, color_picker])])

    out = widgets.interactive_output(plot_rate_perf, 
                                     {'yaxis': yaxis_choice, 
                                      'markers': marker_choice, 
                                      'marker_fill': marker_fill_choice,
                                      'color': color_picker,
                                      'c_e': c_e})
    display(ui, out)

In [ ]:
make_figure()